# Setting up variables

In [13]:
dx = 0.01
dt = 0.01
hcut = 1
m = 1

l = (hcut*dt)/(2*m*dx*dx)
print(l)

50.0


In [14]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [15]:
x = np.round(np.arange(-2,2+dx,dx),3)

In [16]:
v_x = np.zeros(len(x))


In [26]:
main_diag_A = 1+ 1j*l + 1j*dt/(2*hcut)*v_x
off_diag_A = (-1j*l/2)*np.ones(len(x)-1)

diagA = diags([off_diag_A,main_diag_A,off_diag_A],offsets=[-1,0,1],)
print(diagA)

  (np.int32(1), np.int32(0))	-25j
  (np.int32(2), np.int32(1))	-25j
  (np.int32(3), np.int32(2))	-25j
  (np.int32(4), np.int32(3))	-25j
  (np.int32(5), np.int32(4))	-25j
  (np.int32(6), np.int32(5))	-25j
  (np.int32(7), np.int32(6))	-25j
  (np.int32(8), np.int32(7))	-25j
  (np.int32(9), np.int32(8))	-25j
  (np.int32(10), np.int32(9))	-25j
  (np.int32(11), np.int32(10))	-25j
  (np.int32(12), np.int32(11))	-25j
  (np.int32(13), np.int32(12))	-25j
  (np.int32(14), np.int32(13))	-25j
  (np.int32(15), np.int32(14))	-25j
  (np.int32(16), np.int32(15))	-25j
  (np.int32(17), np.int32(16))	-25j
  (np.int32(18), np.int32(17))	-25j
  (np.int32(19), np.int32(18))	-25j
  (np.int32(20), np.int32(19))	-25j
  (np.int32(21), np.int32(20))	-25j
  (np.int32(22), np.int32(21))	-25j
  (np.int32(23), np.int32(22))	-25j
  (np.int32(24), np.int32(23))	-25j
  (np.int32(25), np.int32(24))	-25j
  :	:
  (np.int32(375), np.int32(376))	-25j
  (np.int32(376), np.int32(377))	-25j
  (np.int32(377), np.int32(378))	-25j

In [28]:
main_diag_B = 1- 1j*l - 1j*dt/(2*hcut)*v_x
off_diag_B = (1j*l/2)*np.ones(len(x)-1)

diagB = diags([off_diag_B,main_diag_B,off_diag_B],offsets=[-1,0,1],)
print(diagB.shape)

(401, 401)


## initial gaussian fn